### Read SROIE

In [20]:
import os
import json

In [21]:
def __extract_bbox_and_text(line: str) -> tuple[tuple[int], str]:
    coords = [int(x) for x in line[:8]]
    text = line[8] if len(line) > 8 else ""

    coords = [coords[0], coords[1], coords[0], coords[1]]
    for i in range(0, len(coords), 2):
        coords[0] = min(coords[0], coords[i])
        coords[1] = min(coords[1], coords[i + 1])
        coords[2] = max(coords[2], coords[i])
        coords[3] = max(coords[3], coords[i + 1])
    
    return coords, text

def read_sroie_data(split: str):
    imgs_folder_path = f"../data/sroie/{split}/img"
    labels_folder_path = f"../data/sroie/{split}/entities"

    data = []
    for label_fn in os.listdir(labels_folder_path):
        with open(os.path.join(labels_folder_path, label_fn), "r") as f:
            json_f: dict = json.load(f)

        data.append(dict(
            img_path = os.path.join(imgs_folder_path, label_fn.replace(".txt", ".jpg")),
            label = json_f
        ))

    return data



In [22]:
train_dataset = read_sroie_data("train")
test_dataset = read_sroie_data("test")

### Format Data

In [23]:
from PIL import Image

In [24]:
system_message = """You are a highly advanced Vision Language Model (VLM), specialized in analyzing, describing, and interpreting visual data. 
Your task is to process and extract meaningful insights from images, videos, and visual patterns, 
leveraging multimodal understanding to provide accurate and contextually relevant information."""

In [25]:
train_dataset[0]

{'img_path': '../data/sroie/train/img/X51007135037.jpg',
 'label': {'company': 'S&Y STATIONERY',
  'date': '05-JAN-2017',
  'address': 'NO. 36G JALAN BULAN BM U5/BM, BANDAR PINGGIRAN SUBANG, SEKSYEN U5, 40150 SHAH ALAM, SELANGOR.',
  'total': '72.00'}}

In [26]:
def format_data(sample: dict):
    pil_image = Image.open(sample["img_path"])

    field_names = list(sample["label"].keys())
    output_format = {field: ".." for field in field_names}

    prompt = "Extract the following {fields} from the above document. If a field is not present, return ''. Return the output in a valid JSON format like {output_format}" \
        .format(
            fields = field_names,
            output_format = output_format
        )

    return [
        {
            "role": "system",
            "content": [{"type": "text", "text": system_message}]
        },
        {
            "role": "user",
            "content": [
                { "type": "image", "image": pil_image },
                { "type": "text", "text": prompt }
            ]
        },
        {
            "role": "assistant",
            "content": [{ "type": "text", "text": json.dumps(sample["label"])}]
        }
    ]

In [27]:
train_dataset = [format_data(sample) for sample in train_dataset]
test_dataset = [format_data(sample) for sample in test_dataset]

In [28]:
train_dataset[0]

[{'role': 'system',
  'content': [{'type': 'text',
    'text': 'You are a highly advanced Vision Language Model (VLM), specialized in analyzing, describing, and interpreting visual data. \nYour task is to process and extract meaningful insights from images, videos, and visual patterns, \nleveraging multimodal understanding to provide accurate and contextually relevant information.'}]},
 {'role': 'user',
  'content': [{'type': 'image',
    'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=735x1689>},
   {'type': 'text',
    'text': "Extract the following ['company', 'date', 'address', 'total'] from the above document. If a field is not present, return ''. Return the output in a valid JSON format like {'company': '..', 'date': '..', 'address': '..', 'total': '..'}"}]},
 {'role': 'assistant',
  'content': [{'type': 'text',
    'text': '{"company": "S&Y STATIONERY", "date": "05-JAN-2017", "address": "NO. 36G JALAN BULAN BM U5/BM, BANDAR PINGGIRAN SUBANG, SEKSYEN U5, 40150 SHA

### Training pipeline

In [29]:
import torch
from transformers import Idefics3ForConditionalGeneration, AutoProcessor
from transformers import BitsAndBytesConfig

model_id = "HuggingFaceTB/SmolVLM-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

model = Idefics3ForConditionalGeneration.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)

processor = AutoProcessor.from_pretrained(model_id)

In [19]:
import gc
import time

def clear_memory():
    # Delete variables if they exist in the current global scope
    if "inputs" in globals():
        del globals()["inputs"]
    if "model" in globals():
        del globals()["model"]
    if "processor" in globals():
        del globals()["processor"]
    if "trainer" in globals():
        del globals()["trainer"]
    if "peft_model" in globals():
        del globals()["peft_model"]
    if "bnb_config" in globals():
        del globals()["bnb_config"]
    time.sleep(2)

    # Garbage collection and clearing CUDA memory
    gc.collect()
    time.sleep(2)
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    time.sleep(2)
    gc.collect()
    time.sleep(2)

    print(f"GPU allocated memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU reserved memory: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

clear_memory()

GPU allocated memory: 2.01 GB
GPU reserved memory: 2.78 GB


In [30]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=8,
    lora_alpha=8,
    lora_dropout=0.1,
    target_modules=["down_proj", "o_proj", "k_proj", "q_proj", "gate_proj", "up_proj", "v_proj"],
    use_dora=True,
    init_lora_weights="gaussian",
)

peft_model = get_peft_model(model, peft_config)

peft_model.print_trainable_parameters()

trainable params: 11,269,248 || all params: 2,257,542,128 || trainable%: 0.4992


In [31]:
image_token_id = processor.tokenizer.additional_special_tokens_ids[
    processor.tokenizer.additional_special_tokens.index("<image>")
]


def collate_fn(examples):
    texts = [processor.apply_chat_template(example, tokenize=False) for example in examples]

    image_inputs = []
    for example in examples:
        image = example[1]["content"][0]["image"]
        if image.mode != "RGB":
            image = image.convert("RGB")
        image_inputs.append([image])

    batch = processor(text=texts, images=image_inputs, return_tensors="pt", padding=True)
    labels = batch["input_ids"].clone()
    labels[labels == processor.tokenizer.pad_token_id] = -100  # Mask padding tokens in labels
    labels[labels == image_token_id] = -100  # Mask image token IDs in labels

    batch["labels"] = labels

    return batch

In [32]:
from trl import SFTConfig

# Configure training arguments using SFTConfig
training_args = SFTConfig(
    output_dir="smolvlm-instruct-trl-sft-sroie",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=50,
    learning_rate=1e-4,
    weight_decay=0.01,
    logging_steps=25,
    save_strategy="steps",
    save_steps=25,
    save_total_limit=1,
    optim="adamw_torch_fused",
    bf16=True,
    remove_unused_columns=False,
    gradient_checkpointing=True,
    dataset_text_field="",
    dataset_kwargs={"skip_prepare_dataset": True},
)

In [1]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
    processing_class=processor.tokenizer,
)

/home/rainer/anaconda3/envs/benchmark/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'model' is not defined

In [ ]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 130.00 MiB. GPU 0 has a total capacity of 5.61 GiB of which 35.88 MiB is free. Process 2500 has 67.80 MiB memory in use. Including non-PyTorch memory, this process has 5.09 GiB memory in use. Of the allocated memory 4.89 GiB is allocated by PyTorch, and 127.16 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)